PROGRAM WITH COMMENTS AND EXPLANATION
### PDB CLASSIFICATION BEFORE DOCKING
#### This is a classification program. It can be used to run PDB files and classify them depending on the covalent, non-covalent bonds, and free enzymes. 
In the following code cell we're going to import the libraries. This is the first step to start working with the program.

In [1]:
# Import necessary libraries
import os  # for working with files and directories
import re  # for regular expressions
import csv  # for reading and writing CSV files
import shutil  # for working with files and directories
from Bio.Align import PairwiseAligner  # for pairwise sequence alignment
from Bio.PDB import *  # for working with PDB files

1. Next step, it's to determine the directory. 

It's important to check if the directory is written correctly.
Determine the directory where the PDB files are located. Also, determine where the CSV files and folders will be created.
You can modify the folder "/home/user/folder " to the one where you have saved the PDB files.
Path is where you find information output about the protein.
Directory is where you should save the downloaded protein structures.
Here you can **modify** the folder where the PDB files are located. You can change **path + "folder_with_structures" + "/"**  to the folder that has the structures of interest. It must be written inside " ". Check that the folder name is in red!!

In [2]:
# In this case, my git repository is sort_and_dock, inside this directory there's a folder named as PLpro_structures where all PLpro structures are found
path = "/home/ariadna/pdb/PLpro/"           #modify folder
directory = path + "PLpro_structures" + "/" #modify folder

This line of code creates a list of all files in a specific directory, excluding any other file types or directories using a list comprehension. Get a list of all the PDB files in the directory.

In [4]:
files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

Now, we are going to compare sequences for same PDB structures. In PDB file we can extract three-letter code sequences, so we need to created a one-letter code dictionary in order to ease the alignment.

In [5]:
# Create a dictionary to map amino acid names to single-letter codes
aa_dict = {
    'ALA': 'A',
    'ARG': 'R',
    'ASN': 'N',
    'ASP': 'D',
    'CYS': 'C',
    'GLN': 'Q',
    'GLU': 'E',
    'GLY': 'G',
    'HIS': 'H',
    'HIE': 'H',
    'HID': 'H',
    'ILE': 'I',
    'LEU': 'L',
    'LYS': 'K',
    'MET': 'M',
    'PHE': 'F',
    'PRO': 'P',
    'SER': 'S',
    'THR': 'T',
    'TRP': 'W',
    'TYR': 'Y',
    'VAL': 'V'
}

# Create an empty list to store the amino acid sequences as single-letter codes
a_letter_code = []


The program uses PairWise in Bio library to align sequences, find gaps, find mismatches and to find identity between two structures. The following lines will be used to define how is going to be the alignment.

In [6]:
# Define aligner variables from PairwiseAligner
aligner = PairwiseAligner()
aligner.mode = 'global'
aligner.match_score = 5
aligner.mismatch_score = -3
aligner.open_gap_score = -7
aligner.extend_gap_score = -2

Create a csv named **sequence_SEQRES.csv** where you can read the PDB ID, chain, all residues - in 1-letter code -, and those pseudopeptides that cannot be translate to 1-letter code. 

In [7]:
# Initialize a boolean variable that tracks whether the header row has been written to the CSV file
header_written = False

# Define the path to the CSV file
csv_file_path = path + 'sequence_SEQRES.csv'

# Check if the CSV file exists
if not os.path.isfile(csv_file_path):
    
    # If the CSV file doesn't exist, create it and write the header row
    with open(csv_file_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['PDB_ID', 'Chain_ID', 'Residues', 'Not_in_dict'])

    # Loop over the files and read their contents
    for filename in files:
        with open(os.path.join(directory, filename), "r") as file:
            contents = file.read()

            # Parse the contents of the file and create residues_dict
            residues_dict = {}
            residue_line = ""
            no_dict_matching = []
            
            for pdb_line in contents.splitlines():
                if re.search('^SEQRES .+ . ', pdb_line):
                    pdb_line = re.sub(r'\s+', ' ', pdb_line).strip()
                    chain_id = pdb_line.split(" ")[2] 
                    residue_line_tmp = pdb_line.split(" ")[4:]       
                    residue_line = ' '.join(residue_line_tmp)

                    # Store the residues for this chain in the dictionary
                    if chain_id not in residues_dict:
                        residues_dict[chain_id] = residue_line
                    else:
                        residues_dict[chain_id] += ' ' + residue_line
                
            # Write the data rows to the CSV file if it exists
            with open(csv_file_path, 'a', newline='') as f:
                writer = csv.writer(f)
                
                for chain_id, residues in residues_dict.items():
                    residues_list = residues.split()
                    aa_list = []
                    no_dict_matching = []
                    
                    for residue in residues_list:
                        if residue in aa_dict:
                            aa_list += aa_dict[residue]
                        else:
                            no_dict_matching.append([residue, residues_list.index(residue)])         
                    
                    csv_row = ''.join(aa_list)
                    writer.writerow([filename[3:7], chain_id, csv_row, no_dict_matching])
            

else:
    print(f"{csv_file_path} already exists. Skipping writing to CSV file.")

2. Here we need to choose a structure:
- With no mutation
- In a complex
- Good length (not short, not large)

Write it in reference_structure string. Be careful, and follow the instructions in the comments!

In [8]:
# Here if we want to select one of our batch folder, download directly from the directory we're gonna 
# write pdbXXXX.ent  (depend on ID you choose)
# write ^SEQRES .+ X (depend on the chain you choose)
reference_structure = 'pdb7jiw.ent'
chain_sequence = '^SEQRES .+ A '

In [9]:
# Open pdb file of this reference structure, changing the directory
with open(os.path.join(directory + reference_structure)) as file:
    contents = file.read()

    # seqresA is a temporal variable, in order to secure 3-letter to 1-letter change
    seqres_one_chain = ''

    for pdb_line in contents.splitlines():
        if re.search(chain_sequence, pdb_line):
            pdb_line = re.sub(r'\s+', ' ', pdb_line).strip()
            
            for i in pdb_line.split(' ')[4:]:
                seqres_one_chain = seqres_one_chain + str(aa_dict[i])
                seq1 = seqres_one_chain

Now we're gonna loop over all sequences in the directory to compare with reference sequence.

In [10]:
seq2_list = []
filename_list = []
chain_ID_list = []


with open(csv_file_path, 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)  # skip the header row if it exists
    
    for row in csvreader:
        # do something with the row
        filename_list.append(row[0])
        chain_ID_list.append(row[1])
        seq2_list.append(row[2])

In the following section, the program run the alignments and define those that are good structures and those that have some mutation and need to be revised.

In [14]:
with open(csv_file_path, 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)  # skip the header row if it exists
    
    good_structures = []
    mutations = []
    
    for row in csvreader:
        seq2 = row[2]
        length = len(seq2)

        # Perform pairwise alignment
        alignments = aligner.align(seq1, seq2)
        alignment = alignments[0] # We select the first alignment. It could be more than 1 aligment with same score
        counts = alignment.counts()
        identity = '{:.2f}'.format(alignment.counts()[1]*100/(len(alignment[0, :])))
        
        # Extract the count of mismatches
        mismatches = counts[2]
        gaps = counts[0]

        if mismatches == 0:
            good_structures.append([row[0], row[1], row[2], length, gaps, identity, row[3]])

        if mismatches != 0:
            files_mutations = (row[0], row[1], row[2], row[3])

        # Where is the mismatch
        n = 0
        mismatch_location = []
        
        for i, j in zip(alignment[0, :], alignment[1, :]):
            if i != '-':
                n=n+1
                if i!=j and j!='-':
                    mismatch_location.append(i+str(n)+j)
        
        if len(mismatch_location) != 0:
            mutation_path = path + 'mutation_info.csv'
            mutations.append([files_mutations[0], files_mutations[1], files_mutations[2], length, gaps, mismatches, mismatch_location, identity])
            

In [15]:
# Check if the CSV file exists
if not os.path.isfile(mutation_path):
    
    # Write the header row
    with open(mutation_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['PDB_ID', 'Chain_ID', 'Length', 'Gaps', 'Identity', 'Sequence', 'Mismatches', 'Mismatch_location', 'Not_in_dict'])
        for mutation in mutations:
            writer.writerow(mutation)
            
else:
    print(f"{mutation_path} already exists. Skipping writing to CSV file.")

/home/ariadna/pdb/PLpro/mutation_info.csv already exists. Skipping writing to CSV file.


In [19]:
good_structures_path = path + 'good_structures_info.csv'

# Check if the CSV file exists
if not os.path.isfile(good_structures_path):
    
    # Write the header row
    with open(good_structures_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['PDB_ID', 'Chain_ID', 'Sequence', 'Length', 'Gaps', 'Identity', 'Not_in_dict'])

    with open(good_structures_path, 'a', newline='') as f:
        writer = csv.writer(f)
        for structure in good_structures:
            writer.writerow(structure)
            
else:
    print(f"{good_structures_path} already exists. Skipping writing to CSV file.")

3. You need to *STOP* here in order to check the two csv: "Good_structures" and "Mutation_info". 

You can copy information from mutation_info into good_structures for classify in covalent, non-covalent and free enzymes.

In [20]:
filtered_dir = path + "structures_for_docking/"

In [21]:
PDB_code_list = []
with open(path + 'good_structures_info.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)  # skip the header row if it exists
    for row in csvreader:
        PDB_code_list.append(row[0])


In [22]:
###STRUCTURES_FOR_DOCKING
if not os.path.exists(filtered_dir):
    os.makedirs(filtered_dir)

target = filtered_dir
origin = directory  

# Fetching the list of all the files
files = os.listdir(origin)

# Fetching all the files to directory
for name in PDB_code_list:
    shutil.copy(origin+("pdb"+ name+ ".ent"), target+("pdb"+ name + ".ent"))

In [23]:
filtered_files = [f for f in os.listdir(filtered_dir) if os.path.isfile(os.path.join(filtered_dir, f))]

******** 

Now, we set the blacklist.txt directory. It shouldn't be modified.
To keep the blacklist up-to-date, specify the directory of the text file with the codes that appear in column 8-10 on the HET line.

In [25]:
blacklist = "/home/ariadna/2023/PROTIC/blacklist.txt"

with open(blacklist, 'r') as f:
    linea = f.read()
linea=linea.replace(' ','')
blacklist_ID = linea.strip('\n').split(',')

4. Here, we can also **modify** the residue or the covalent bond we want to look for, changing the string. 

The "." character means that any character could be in the same position. So, for example, in PDB files, ATOM lines are *residue + chain + number of the residue*. You also need to keep the whitespaces in order to get the match. Also, the "^" character means that the line starts with the following. For example, ^*LINK* means that the line starts with LINK.

Check that the string is in red!!

In [26]:
res_1 = "CYS . 111 " #modify this residue
link = "^LINK .+ CYS . 111 " #modify the residue of covalent bond, and if you don't know, just leave "^LINK .+ "

Now, we can find the core of the code. This it shouldn't be modified.

In [27]:
# Define the three classification lists that we want to obtain.

covalent_list = []
free_enzymes_list = []
non_covalent_list = []

# Define the string of information that we want to add to the CSV.

information_covalent=""
information_free=""
information_non_covalent=""

# Open a for loop to iterate through all the PDB files in the mentioned directory
# Read the files using .decode() for Ubuntu and its encoding reasons.

for pdb_file in filtered_files:
    with open(os.path.join(filtered_dir, pdb_file), "rb") as f:
        pdb_info = f.read().decode()
        
    # Here we will define the variables that will help to perform the 
    # classification. 'no_bond' refers to having no bond;  'covalent' refers to having a covalent bond; 'in_black' if it corresponds to an element in the blacklist; 'remove' if it is necessary
    # to remove the file from the classification; 'possible_noncovalent' to recognize structures with the potential to be
    # non-covalent.

    no_bond = 1
    covalent = 0
    in_black = 1
    remove = 1
    possible_noncovalent = 1

    # Open a for loop to iterate through all the lines of each PDB file and propose the conditions.
    for line in pdb_info.splitlines():
        if re.search(res_1, line):
            remove = 0

        if re.search(link, line):
            line = re.compile(r'(\s)\1{1,}').sub(r'\1', line)
            
            # Check that the HET_ID appears at position 5, sometimes it appears at position 6!
            if line.upper().split(' ')[6] not in blacklist_ID:
                covalent = 1
                information_covalent = line
                break
            
        if re.search("^HET ", line):
            no_bond = 0
            line = re.compile(r'(\s)\1{1,}').sub(r'\1', line)
            separate_lines = line.split(" ")
            separate_lines = separate_lines[0:5]
            information_free= line
            
            if not separate_lines[1] in blacklist_ID:
                in_black = 0
                information_non_covalent = line


   # End of the for loop that iterates through the lines of the file.                
    if remove == 1:
        print(pdb_file, "mutada")
    elif covalent == 1:
        covalent_list.append([pdb_file[3:7], information_covalent])
    elif no_bond == 0 and in_black == 0:
        non_covalent_list.append([pdb_file[3:7], information_non_covalent])
    else:
        free_enzymes_list.append([pdb_file[3:7], information_free])

print(len(covalent_list), len(free_enzymes_list), len(non_covalent_list))

pdb7qcm.ent mutada
pdb7qcj.ent mutada
pdb7qch.ent mutada
pdb7qci.ent mutada
pdb7qck.ent mutada
pdb7qcg.ent mutada
pdb7nt4.ent mutada
5 3 16


This part of the code writes three csv files, one for each group. You can edit the csv name, replacing 'DataCovalent.csv' for another one. **Keep writing .csv** !!!!

In [222]:
def write_csv_for_each_group(name_of_csv, type_list):
    fields = ['PDB ID', 'LINE']
    rows = type_list[:]
    with open(path + name_of_csv, 'w') as f:
        csv_writer = csv.writer(f)
        csv_writer.writerow(fields)
        csv_writer.writerows(rows)
        
write_csv_for_each_group('DataCovalent.csv', covalent_list)
write_csv_for_each_group('DataNonCovalent.csv', non_covalent_list)
write_csv_for_each_group('DataFree.csv', free_enzymes_list)

CREATE A NEW FOLDER TO SAVE THE FILES OF EACH GROUP

1. Free_Folder
2. NonCovalent_Folder
3. Covalent_Folder

In [223]:
###FREE PROTEINS
free_list_ID=[item[0] for item in free_enzymes_list]

newpath = path + 'Free_Folder/'
if not os.path.exists(newpath):
    os.makedirs(newpath)

target = newpath
origin = filtered_dir

# Fetching the list of all the files
files = os.listdir(origin)

# Fetching all the files to directory
for name in free_list_ID:
    shutil.copy(origin+("pdb"+ name+ ".ent"), target+("pdb"+ name + ".ent"))
    
    
###NON-COVALENT PROTEINS

non_covalent_list_ID=[item[0] for item in non_covalent_list]

newpath = path + 'NonCovalent_Folder/'
if not os.path.exists(newpath):
    os.makedirs(newpath)

target = newpath
# Fetching the list of all the files
files = os.listdir(origin)

# Fetching all the files to directory
for pdb_file in non_covalent_list_ID:
    shutil.copy(origin+("pdb"+ pdb_file + ".ent"), target+("pdb"+ pdb_file + ".ent"))


###COVALENT PROTEINS

covalent_list_ID=[item[0] for item in covalent_list]

newpath = path + 'Covalent_Folder/'
if not os.path.exists(newpath):
    os.makedirs(newpath)

target = newpath

# Fetching the list of all the files
files = os.listdir(origin)

# Fetching all the files to directory
for pdb_file in covalent_list_ID:
    shutil.copy(origin+("pdb"+ pdb_file + ".ent"), target+("pdb"+ pdb_file + ".ent"))